캐글 과제 설명
**제출 파일
테스트 세트의 각 key_id에 대해 최대 3개의 워드 값을 예측해야 한다. 파일에는 헤더가 포함되어야 하며 다음과 같은 형식이 있어야 한다 중요: 어떤 "단어"는 한 단어 이상 교육 데이터는 이전에 출시된 Quick Draw 데이터 세트와 정렬되며, 공백으로 다중 단어 레이블을 구분한다. 이 경기의 카글 메트릭에는 공백이 없는 라벨이 필요하므로 공간을 밑줄로 대체하기 위해 라벨 예측을 조정해야 한다. 예를 들어, "롤러 코스터"는 "롤러_코스터"로 예측해야 한다.**

기존 Quick, Draw! 데이터 세트에 대한 더 나은 분류자를 만드는 것이 과제이다. Kaggler는 이 데이터 세트에서 모델을 발전시킴으로써 패턴 인식 솔루션을 보다 광범위하게 개선할 수 있다. 

**import 필요 모듈**
데이터 불러오기 및 확인

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import ast
import os
from glob import glob
from tqdm import tqdm
from dask import bag
from PIL import Image, ImageDraw 
import cv2
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, DepthwiseConv2D, BatchNormalization, ZeroPadding2D
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, AveragePooling2D 
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.metrics import top_k_categorical_accuracy
import os
from tqdm import tqdm #for문 처리과정 확인 함수 

In [ ]:
rd=np.random.randint(340)#340까지의 수 중에 난수생성
path = os.listdir('/kaggle/input/quickdraw-doodle-recognition/train_simplified') 
# df = pd.read_csv('/kaggle/input/quickdraw-doodle-recognition/train_simplified/'+path[rd])
# print (df)

open_cv를 이용 이미지 그려주기


In [ ]:
height = 64
width = 64

In [ ]:
def draw_to_img(strokes):
    BASE_SIZE = 256
    image = Image.new("P", (BASE_SIZE,BASE_SIZE), color=255) 
    image_draw = ImageDraw.Draw(image)
    for each in ast.literal_eval(strokes):
        for i in range(len(each[0])-1):
            p1 = each[0][i],each[1][i]
            p2 = each[0][i+1],each[1][i+1]
            image_draw.line([p1,p2],width=5)
    img = image.resize((height,width))
    return np.array(img)/255.

word에 띄어쓰기  '_' 로변경하기 
tmp설정후 이미지확인

In [ ]:
# tmp = df['drawing'][20]
# tmp2 = df['word'][20]
# df['word']= df['word'].replace(' ','_',regex = True)
# img = np.array(draw_to_img(tmp))
# #데이터 형태확인
# print(tmp)
# print(' ')
# print(tmp2)
# plt.imshow(img)



In [ ]:
#test
ran=np.random.randint(340)#340까지의 수 중에 난수생성
nums2names={i : v[:-4].replace(' ','_') for i , v in enumerate(path)}#불러온 데이터에 index 번호붙이기
ranclass=nums2names[ran]# 340까지의 수중에 랜덤으로 불러오기  
ranclass=ranclass.replace('_',' ')# 분류 제목을 _을 ' '로 바꿔주기
rdpath='/kaggle/input/quickdraw-doodle-recognition/train_simplified/'+ranclass+'.csv' #랜덤으로 하나의 클래스 경로설정
one=pd.read_csv(rdpath,usecols=['drawing','recognized','word'],nrows=10) #10개 행의 drawing recognized word 불러오기
one=one[one.recognized==True].head(2)#그림 중 true인것 2개만불러오기
name=one['word'].head(1)#one의 첫번째 word 
strk=one['drawing']# one 의 drawing #2개
name=name.values


데이터 추가 하기 

In [ ]:
train_grand=[]
num_class = 340
per_class = 800

In [ ]:
class_paths = glob('/kaggle/input/quickdraw-doodle-recognition/train_simplified/*.csv')
for i , c in enumerate(tqdm(class_paths[0:num_class])): 
    train=pd.read_csv(c,usecols=['drawing','recognized'],nrows=per_class*1.5)
    train=train[train.recognized==True].head(per_class)
    imagebag=bag.from_sequence(train.drawing.values).map(draw_to_img)
    train_array=np.array(imagebag.compute())#unmpy 형식
    train_array=np.reshape(train_array,(per_class,-1))  #2000  -1로 reshpae로 행렬 형식 변경 그림형식 
    label_array=np.full((train.shape[0],1),i)# label 붙여주기 , train.shape[0] = 2000
    train_array=np.concatenate((label_array,train_array),axis=1)
    train_grand.append(train_array)
    del train_array
    del label_array


In [ ]:
train_grand=np.array([train_grand.pop() for i in np.arange(num_class)]) #데이터 세팅

In [ ]:
train_grand=train_grand.reshape((-1,(height*width+1))) #64*64로 배열 변경
print(train_grand)

setting to ( training set :  test set ) =  ( 9 : 1  )

In [ ]:
specific = 0.1 
sequence_length = 50
cut = int(specific * train_grand.shape[0])
print(cut)

np.random.shuffle(train_grand)
y_train, X_train = train_grand[cut: , 0], train_grand[cut: , 1:]
y_val, X_val = train_grand[0:cut, 0], train_grand[0:cut, 1:]

# del train_grand

x_train=X_train.reshape(X_train.shape[0],height,width,1)
x_val=X_val.reshape(X_val.shape[0],height,width,1)


y_train2 = keras.utils.to_categorical(y_train, num_class)


print(y_train.shape, "\n",
      x_train.shape, "\n",
      y_val.shape, "\n",
      x_val.shape)



In [ ]:
leaky_relu = tf.nn.leaky_relu

In [ ]:
model = Sequential()
model.add(ZeroPadding2D(padding=(1, 1),input_shape=(height,width,1)))
model.add(Conv2D(32,kernel_size=(3,3),padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2))) 
model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(ZeroPadding2D(padding=(1, 1)))
model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation=leaky_relu))
model.add(AveragePooling2D(pool_size=(2,2)))
model.add(ZeroPadding2D(padding=(1, 1)))
model.add(Activation('relu'))
model.add(DepthwiseConv2D(64, padding='same',activation='relu' ))
model.add(AveragePooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(128,kernel_size=(3,3),padding='same',activation='relu'))
model.add(Flatten())
model.add(Dense(num_class*5, activation='relu'))
model.add(Dense(num_class,activation='softmax'))
model.summary()

In [ ]:
def top_3_accuracy(x, y):
    return top_k_categorical_accuracy(x, y, k=3)

In [ ]:
reduceLROnPlat=ReduceLROnPlateau(monitor='val_loss',factor=0.5,patience=3,
                                 verbose=1,mode='auto',min_delta=0.005,
                                 cooldown=5,min_lr=0.0001)
callbacks=[reduceLROnPlat]
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',
              metrics=['accuracy',top_3_accuracy])

history=model.fit(x=x_train,y=y_train,batch_size=32,epochs=20,
                  validation_data=(x_val,y_val),callbacks=callbacks,verbose=1)

In [ ]:
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss= history.history['loss']
val_loss=history.history['val_loss']

epochs=range(1,len(acc)+1)

plt.plot(epochs,acc,label='Training acc')
plt.plot(epochs,val_acc,label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs,loss,label='Training loss')
plt.plot(epochs,val_loss,label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
list=[]
reader=pd.read_csv('/kaggle/input/quickdraw-doodle-recognition/test_simplified.csv',index_col=['key_id'],chunksize=2048)
for chunk in tqdm(reader,total=55):
    imagebag=bag.from_sequence(chunk.drawing.values).map(draw_to_img)
    testarray=np.array(imagebag.compute())
    testarray=np.reshape(testarray,(testarray.shape[0],height,width,1))
    testpreds=model.predict(testarray,verbose=0)
    s=np.argsort(-testpreds)[:,0:3]
    list.append(s)
array=np.concatenate(list)
pred_df=pd.DataFrame({'first': array[:,0],'second':array[:,1],'third':array[:,2]})
pred_df=pred_df.replace(nums2names)
pred_df['words']=pred_df['first']+' '+pred_df['second']+' '+pred_df['third']

sub=pd.read_csv('/kaggle/input/quickdraw-doodle-recognition/sample_submission.csv',index_col=['key_id'])
sub['word']=pred_df.words.values
sub.to_csv('result_of_mission.csv')

In [ ]:
sub.head()